In [1]:
"""
Created on Tue Feb 12 23:34:44 2019
@author: tobia
"""
import time
import datetime
import urllib.request, json

import numpy as np

import pandas as pd
from datetime import datetime, timedelta
from oceanoobsbrasil.db import GetData

from itertools import cycle
from oceanoobsbrasil.utils import *

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec
from selenium.webdriver.chrome.options import Options
import chromedriver_binary

from dotenv import load_dotenv
import os
import requests
import warnings
warnings.filterwarnings("ignore")


In [2]:
# import geckodriver_autoinstaller
# geckodriver_autoinstaller.install()

In [3]:
%load_ext autoreload
%autoreload 2
from oceanoobsbrasil.weather_stations.inmet import Inmet
from oceanoobsbrasil.buoys.pnboia import Pnboia

In [4]:

import psutil
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import time
import datetime
from selenium.webdriver.common.action_chains import ActionChains
import operator
import requests

import pandas as pd
import numpy as np

from pandas.io import sql
import sqlalchemy

In [5]:
i = Inmet()

In [ ]:
i.get()

A110
https://tempo.inmet.gov.br/TabelaEstacoes/A110
XXXXXXXXXXXX
XXXXXXXXXXXX
    atmp    pres  wspd   wdir  gust           date_time
0   27.4   997.3   2.9   97.0   6.4 2024-01-21 03:00:00
1   27.1   998.5   1.4  292.0   5.4 2024-01-21 04:00:00
2   26.8   999.4   0.0   74.0   4.7 2024-01-21 05:00:00
3   26.2   999.3   2.7  284.0   5.2 2024-01-21 06:00:00
4   25.5   999.3   1.0  177.0   4.5 2024-01-21 07:00:00
5   25.5   999.0   0.0   64.0   2.9 2024-01-21 08:00:00
6   25.6   999.1   1.2   14.0   3.5 2024-01-21 09:00:00
7   25.5   998.8   1.0  335.0   4.5 2024-01-21 10:00:00
8   25.3   998.6   1.4  346.0   5.2 2024-01-21 11:00:00
9   25.2   998.7   2.9    5.0   6.4 2024-01-21 12:00:00
10  24.8   999.4   0.4   12.0   6.4 2024-01-21 13:00:00
11  24.9  1000.3   0.0  343.0   2.9 2024-01-21 14:00:00
12  25.6  1001.1   0.0  159.0   1.4 2024-01-21 15:00:00
13  26.6  1001.7   1.6  323.0   4.3 2024-01-21 16:00:00
14  28.0  1001.9   2.7  307.0   6.8 2024-01-21 17:00:00
15  28.5  1001.8   2.1  25

In [18]:
url = "https://tempo.inmet.gov.br/TabelaEstacoes/A212"

In [16]:
options = webdriver.FirefoxOptions()
options.add_argument('-headless')
#options.add_argument("--no-sandbox");
options.add_argument("--disable-dev-shm-usage");
options.set_preference("dom.disable_beforeunload", True)
options.set_preference("browser.tabs.warnOnClose", False)

In [17]:
print("conectando ao site do inmet")
driver = webdriver.Firefox(options=options)
print("conectado com sucesso")
driver.get(url)
time.sleep(8)

conectando ao site do inmet


KeyboardInterrupt: 

In [104]:
df = pd.read_html(driver.page_source, decimal=',', thousands='.')

In [110]:
url

'https://tempo.inmet.gov.br/TabelaEstacoes/A212'

In [109]:
df[0].isna().sum().sum()

408

In [91]:
df = pd.read_html(driver.page_source, decimal=',', thousands='.')
df = df[0].iloc[:,[0,1, 2, 11, 14, 15, 16]]
df.columns = ['date', 'hour', 'atmp', 'pres', 'wspd', 'wdir', 'gust']

In [92]:
df['hour'] = (df['hour']/100).astype(int).astype('str').str.zfill(2)
df['date_time'] = pd.to_datetime(df['date'] + df['hour'], format='%d/%m/%Y%H')


In [93]:
df.drop(columns=['date', 'hour'], inplace=True)

In [94]:
result = df.replace(to_replace =['None', 'NULL', ' ', ''],
                        value =np.nan)

result.date_time = result.date_time + timedelta(hours=3)
result = result[result.date_time <datetime.datetime.utcnow()]

result['gust'] = pd.to_numeric(result['gust'], errors='coerce')
result.gust[result.gust.notnull()] = (result.gust[result.gust.notnull()]*1.94384).round(decimals=1)
result['wspd'] = pd.to_numeric(result['wspd'], errors='coerce')
result.wspd[result.wspd.notnull()] = (result.wspd[result.wspd.notnull()]*1.94384).round(decimals=1)


In [95]:
result

,atmp,pres,wspd,wdir,gust,date_time
0,19.2,888.2,2.5,204.0,7.2,2023-02-22 03:00:00
1,19.7,888.9,2.1,158.0,5.6,2023-02-22 04:00:00
2,19.7,889.4,2.5,154.0,4.7,2023-02-22 05:00:00
3,19.4,889.3,3.5,171.0,8.0,2023-02-22 06:00:00
4,18.9,888.8,2.9,190.0,8.0,2023-02-22 07:00:00
5,18.8,888.3,2.5,192.0,8.4,2023-02-22 08:00:00
6,17.3,888.8,7.8,103.0,14.2,2023-02-22 09:00:00
7,17.0,888.7,5.4,110.0,13.2,2023-02-22 10:00:00
8,17.1,889.1,3.9,136.0,10.3,2023-02-22 11:00:00
9,16.9,889.1,1.6,270.0,7.0,2023-02-22 12:00:00


In [22]:
i = Inmet()

In [23]:
url = "https://tempo.inmet.gov.br/TabelaEstacoes/A110"

In [24]:
i.driver = webdriver.Chrome(options=i.options)
i.driver.get(url)

In [27]:
BeautifulSoup(i.driver.page_source, 'lxml')

UnexpectedAlertPresentException: Alert Text: Erro ao carregar lista de estações.
Message: unexpected alert open: {Alert text : Erro ao carregar lista de estações.}
  (Session info: headless chrome=110.0.5481.100)
Stacktrace:
#0 0x562112b97d93 <unknown>
#1 0x5621129662d7 <unknown>
#2 0x5621129deb5b <unknown>
#3 0x5621129c6353 <unknown>
#4 0x562112995e40 <unknown>
#5 0x562112997038 <unknown>
#6 0x562112beb8be <unknown>
#7 0x562112bef8f0 <unknown>
#8 0x562112bcff90 <unknown>
#9 0x562112bf0b7d <unknown>
#10 0x562112bc1578 <unknown>
#11 0x562112c15348 <unknown>
#12 0x562112c154d6 <unknown>
#13 0x562112c2f341 <unknown>
#14 0x7f67d4378b43 <unknown>


In [21]:
pd.read_html(i.driver.page_source, decimal=',', thousands='.')

UnexpectedAlertPresentException: Alert Text: Erro ao carregar lista de estações.
Message: unexpected alert open: {Alert text : Erro ao carregar lista de estações.}
  (Session info: headless chrome=110.0.5481.100)
Stacktrace:
#0 0x55ab0046dd93 <unknown>
#1 0x55ab0023c2d7 <unknown>
#2 0x55ab002b4b5b <unknown>
#3 0x55ab0029c353 <unknown>
#4 0x55ab0026be40 <unknown>
#5 0x55ab0026d038 <unknown>
#6 0x55ab004c18be <unknown>
#7 0x55ab004c58f0 <unknown>
#8 0x55ab004a5f90 <unknown>
#9 0x55ab004c6b7d <unknown>
#10 0x55ab00497578 <unknown>
#11 0x55ab004eb348 <unknown>
#12 0x55ab004eb4d6 <unknown>
#13 0x55ab00505341 <unknown>
#14 0x7f3efa7aeb43 <unknown>


In [14]:
i.get(save_bd=True)

A110
Alert Text: Erro ao carregar lista de produtos.
Message: unexpected alert open: {Alert text : Erro ao carregar lista de produtos.}
  (Session info: headless chrome=110.0.5481.100)
Stacktrace:
#0 0x563bf7757d93 <unknown>
#1 0x563bf75262d7 <unknown>
#2 0x563bf759eb5b <unknown>
#3 0x563bf7586353 <unknown>
#4 0x563bf7555e40 <unknown>
#5 0x563bf7557038 <unknown>
#6 0x563bf77ab8be <unknown>
#7 0x563bf77af8f0 <unknown>
#8 0x563bf778ff90 <unknown>
#9 0x563bf77b0b7d <unknown>
#10 0x563bf7781578 <unknown>
#11 0x563bf77d5348 <unknown>
#12 0x563bf77d54d6 <unknown>
#13 0x563bf77ef341 <unknown>
#14 0x7fbe2a56ab43 <unknown>

Nao ha dados para essa estação
driver is running
chrome is still running, we can quit
driver has died
A013
Alert Text: Erro ao carregar lista de produtos.
Message: unexpected alert open: {Alert text : Erro ao carregar lista de produtos.}
  (Session info: headless chrome=110.0.5481.100)
Stacktrace:
#0 0x55a800e79d93 <unknown>
#1 0x55a800c482d7 <unknown>
#2 0x55a800cc0b5b <unk

KeyboardInterrupt: 

In [5]:
start_date=(datetime.utcnow() - timedelta(days=1)).strftime('%Y-%m-%d')
end_date = (datetime.utcnow() + timedelta(days=0)).strftime('%Y-%m-%d')
api="https://apitempo.inmet.gov.br"

In [6]:
db = GetData()
stations = db.get(table='stations', institution=['=', 'inmet'])

In [7]:
stations

,id,name,lat,lon,data_type,institution,url,flag
0,341,BOCA DO ACRE,-8.77680,-67.3326,meteorological_station,inmet,A110,True
1,293,ARAGARCAS,-15.90270,-52.2452,meteorological_station,inmet,A013,True
2,301,ARAGUAINA,-7.10395,-48.2012,meteorological_station,inmet,A021,True
3,303,CAIAPONIA,-16.96690,-51.8175,meteorological_station,inmet,A023,True
4,412,ALTAMIRA,-3.27028,-52.3947,meteorological_station,inmet,A253,True
...,...,...,...,...,...,...,...,...
584,864,IGUATEMI,-23.64450,-54.5703,meteorological_station,inmet,S709,True
585,865,ITAPORA,-22.09280,-54.7988,meteorological_station,inmet,S710,True
586,866,LAGUNA CARAPA,-22.57540,-55.1603,meteorological_station,inmet,S711,True
587,867,NOVA ALVORADA DO SUL,-21.45100,-54.3420,meteorological_station,inmet,S712,True


In [10]:
url = f"{api}/estacao/{start_date}/{end_date}/A212"
url

'https://apitempo.inmet.gov.br/estacao/2023-02-21/2023-02-22/A212'

In [12]:
response = requests.get(url)

In [11]:
response = requests.get(url).json()

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [41]:
df = pd.DataFrame(response)
df = df[['PRE_INS', 'PTO_INS', 'VEN_DIR', 'DT_MEDICAO', 'CHUVA', 'VEN_VEL','VEN_RAJ', 'TEM_INS',
         'UMD_INS', 'HR_MEDICAO']]
df.columns = ['pres', 'dewp', 'wdir', 'date', 'prec', 'wspd', 'gust', 'atmp', 'humi', 'hour']
df['date_time'] = pd.to_datetime(df['date'] + df['hour'], format='%Y-%m-%d%H%M')
df.drop(columns=['date', 'hour'], inplace=True)
df = df[df.date_time <datetime.utcnow()]

In [42]:
df.tail(40)

,pres,dewp,wdir,prec,wspd,gust,atmp,humi,date_time
0,886.7,16.4,312,0,1.5,3.4,22.2,70,2021-10-16 00:00:00
1,888.1,15.9,339,0,1.9,3.1,22.4,67,2021-10-16 01:00:00
2,888.3,15.7,331,0,1.8,4.5,22.2,67,2021-10-16 02:00:00
3,887.5,15.8,344,0,2.3,4.4,22.5,66,2021-10-16 03:00:00
4,887.1,16.2,336,0,2.5,5,21.5,72,2021-10-16 04:00:00
5,886.9,16.3,311,0,1.3,5,20.1,79,2021-10-16 05:00:00
6,886.2,16.3,320,0,1.3,2.7,20,79,2021-10-16 06:00:00
7,886.6,16.6,312,0,1.4,2.1,20.6,78,2021-10-16 07:00:00
8,887.2,17,39,0,0.2,2.5,20.3,81,2021-10-16 08:00:00
9,888,17.5,45,0,0.5,0.8,19.9,86,2021-10-16 09:00:00


In [32]:
df.date_time<datetime.utcnow()

0      True
1      True
2      True
3      True
4      True
      ...  
67    False
68    False
69    False
70    False
71    False
Name: date_time, Length: 72, dtype: bool

0     2021-10-160000
1     2021-10-160100
2     2021-10-160200
3     2021-10-160300
4     2021-10-160400
           ...      
67    2021-10-181900
68    2021-10-182000
69    2021-10-182100
70    2021-10-182200
71    2021-10-182300
Length: 72, dtype: object

In [21]:
df

,pres,dewp,wdir,date,prec,wspd,gust,atmp,humi,hour
0,885.3,18.9,79,2021-10-14,0,0.8,3.4,21.9,83,0000
1,885.5,18.7,89,2021-10-14,0,2.6,5.2,22,81,0100
2,885.2,18.6,45,2021-10-14,0,1.4,5.2,21.8,82,0200
3,884.9,18.5,36,2021-10-14,0,0.8,3.7,21.1,85,0300
4,884.5,18.6,3,2021-10-14,0,1.7,2.8,21.1,86,0400
...,...,...,...,...,...,...,...,...,...,...
115,None,None,None,2021-10-18,None,None,None,None,None,1900
116,None,None,None,2021-10-18,None,None,None,None,None,2000
117,None,None,None,2021-10-18,None,None,None,None,None,2100
118,None,None,None,2021-10-18,None,None,None,None,None,2200


In [12]:
p = Pnboia()

In [13]:
p.get()

,date_time,lat,lon,rh,pres,atmp,dewpt,wspd,wdir,gust,arad,sst,swvht,mxwvht,tp,wvdir
0,2021-10-14 00:00:00,-25.5146,-42.7257,80.5,1018.1,22.0,18.5,0.45,183,14.74,0.03,21.96,3.30,4.77,8.7,106
1,2021-10-14 01:00:00,-25.5144,-42.7257,80.7,1017.5,21.9,18.6,1.25,123,15.42,0.02,21.94,3.36,5.32,8.3,97
2,2021-10-14 03:00:00,-25.5135,-42.7252,80.7,1016.3,21.9,18.7,0.34,242,14.74,0.02,21.89,3.22,5.27,9.1,95
3,2021-10-14 04:00:00,-25.5129,-42.7251,80.3,1015.8,21.9,18.5,0.68,189,14.74,0.04,21.87,3.25,4.93,8.7,88
4,2021-10-14 05:00:00,-25.5122,-42.7248,81.9,1015.3,21.8,18.4,0.79,191,15.87,0.02,21.86,3.14,4.91,9.1,93
5,2021-10-14 06:00:00,-25.5113,-42.7246,81.6,1014.6,21.8,18.2,0.79,96,13.94,0.04,21.85,3.15,5.11,9.1,85
6,2021-10-14 07:00:00,-25.5102,-42.7245,82.5,1014.6,21.7,18.6,0.91,182,13.72,0.00,21.82,2.95,4.30,8.3,95
7,2021-10-14 08:00:00,-25.5091,-42.7244,85.2,1014.8,21.6,19.1,1.02,169,11.68,0.03,21.80,3.17,6.13,8.7,89
8,2021-10-14 09:00:00,-25.5082,-42.7245,84.3,1015.6,21.7,19.1,0.57,183,11.22,0.08,21.78,3.14,4.52,9.1,81
9,2021-10-14 10:00:00,-25.5079,-42.7246,84.7,1015.9,21.8,19.3,0.91,178,12.24,0.08,21.77,3.57,5.06,8.7,80


In [4]:
bd = GetData()

In [4]:
bd.get(table='stations', institution=['=', 'simcosta'], data_type=['=', 'buoy'])

,id,name,lat,lon,data_type,institution,url
0,897,RJ3,-22.9831,-43.1745,buoy,simcosta,3
1,898,RS2,-32.1346,-52098.0000,buoy,simcosta,6
2,899,RS5,-32296.0000,-52.0239,buoy,simcosta,10
3,900,RJ4,-22.9715,-43.1522,buoy,simcosta,12
4,901,RS3,-32195.0000,-52.0794,buoy,simcosta,13
5,902,RS4,-32.2454,-52.0954,buoy,simcosta,14
6,903,BA1,-12.9895,-37.4582,buoy,simcosta,15


In [8]:
df = bd.get(table='data_stations', station_id=['in', [902]])

In [9]:
df.shape

(94, 22)

In [7]:
bd.post(table='data_stations', df=df)

DELETE FROM data_stations WHERE true AND station_id in (902) AND date_time >= '2021-09-06 00:21:00'


In [3]:
import time
import datetime
import operator
import urllib.request, json

import numpy as np
from oceanoobsbrasil.buoys.simcosta import Simcosta
from oceanoobsbrasil.tides.simcosta import SimcostaTide

In [4]:
s = SimcostaTide()

In [5]:
s.get()

Nao ha dados para essa boia
Nao ha dados para essa boia
Nao ha dados para essa boia


KeyboardInterrupt: 

In [52]:
s = Simcosta()

In [53]:
s.get()

id                  897
name                RJ3
lat            -22.9831
lon            -43.1745
data_type          buoy
institution    simcosta
url                   3
Name: 0, dtype: object
http://simcosta.furg.br/api/metereo_data?boiaID=3&type=json&time1=1631069870&time2=1631156270&params=Average_wind_direction_N,Last_sampling_interval_gust_speed,Average_Dew_Point,Average_Pressure,Solar_Radiation_Average_Reading,Average_Air_Temperature,Instantaneous_Humidity,Average_Humidity,Average_wind_speed
http://simcosta.furg.br/api/oceanic_data?boiaID=3&type=json&time1=1631069870&time2=1631156270&params=H10,HAvg,Hsig_Significant_Wave_Height_m,HM0,Mean_Wave_Direction_deg,Hmax_Maximum_Wave_Height_m,ZCN,Tp5,TAvg,T10,Tsig,Mean_Spread_deg,TP_Peak_Period_seconds,Average_Salinity,Average_Temperature_deg_C,Average_Temperature_C,Average_CDOM_QSDE,Average_Chlorophyll_Fluorescence,Average_Dissolved_Oxygen,Average_Nephelometric_Turbidity_Unit_NTU,Cell_Average_Direction_N,Cell_Average_Magnitude_mm_s
Empty D

In [40]:
                columns = ['YEAR','MONTH','DAY','HOUR','MINUTE','SECOND','Hmt',
                    'Avg_Wnd_Dir','M_Decl','Avg_W_Tmp1','Avg_Sal','Avg_Spre_N',
                    'Avg_Wv_Dir','Avg_Cel1_Mag','Avg_Cel1_Dir','Avg_Cel1_Dir_N',
                    'Avg_Turb','Avg_Chl','Avg_DO','ZCN','HM0','TAvg','Tp5',
                    'T10','HAvg','Tsig','CDOM','H10','Avg_Sol_Rad']



In [43]:
s.result.drop(columns=columns, inplace=True)

In [44]:
s.result.columns = ['pres', 'atmp', 'rh', 'dewpt', 'wspd', 'wdir', 'gust', 'swvht', 'mxwvht', 'tp', 'sst', 'wvspread', 'wvdir', 'date_time']

In [45]:
s.result = s.result.replace(to_replace =['None', 'NULL', ' ', ''],
                                        value =np.nan)

In [46]:
s.result['station_id'] = 898

In [ ]:
s.